In [6]:
# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage, ToolMessage
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime

from langgraph.types import Command


import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
model_name = os.getenv("MODEL_NAME")


# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)
from typing import TypedDict


class State(AgentState):
    test: str


@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取指定城市的天气信息
    Args:
        city (str): 城市名称
    """
    # 这里可以集成实际的天气API调用
    test = runtime.state.get("test")
    print("1:", test)

    runtime.state.update({"test": 797})

    return f"{city}的天气是晴朗，温度25摄氏度。"


@tool
def get_xz(xz: str, runtime: ToolRuntime) -> str:
    """获取星座的今日运势

    Args:
        xz (str): 星座

    Returns:
        str: 对应星座的今日运势
    """
    test = runtime.state.get("test", "1")
    print("2:", test)

    return f"{xz}的今日运势：桃花"


bi_agent = create_agent(model=model, tools=[get_weather, get_xz], state_schema=State)

result = bi_agent.invoke(
    {
        "messages": [{"role": "user", "content": "先看下上海天气如何,然后再看下双子座今日运势如何"}],
    }
)

print(result)

1: None
2: 797
{'messages': [HumanMessage(content='先看下上海天气如何,然后再看下双子座今日运势如何', additional_kwargs={}, response_metadata={}, id='fc95996f-c18b-4151-af34-9db578b5329d'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 366, 'total_tokens': 412, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-198de28e-bf95-42d6-b629-22703c95c917', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f5fbb05b-4396-48a8-9cca-f1f3a68525ee-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '上海'}, 'id': 'call_0768fddec1f64c8d804973a9', 'type': 'tool_call'}, {'name': 'get_xz', 'args': {'xz': '双子座'}, 'id': 'call_9d2d4970af8e4c778ec08289', 'type': 'tool_call'}], usage_metadata={'input_tokens': 366, 'output_tokens': 46, 'total_tokens': 412, 'input_token_details':